# https://www.tensorflow.org/get_started/get_started

In [38]:
3 # a rank 0 tensor; this is a scalar with shape []

3

In [39]:
[1. ,2., 3.] # a rank 1 tensor; this is a vector with shape [3]

[1.0, 2.0, 3.0]

In [40]:
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

In [41]:
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

In [42]:
import tensorflow as tf

In [43]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [44]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_4:0", shape=(), dtype=float32) Tensor("Const_5:0", shape=(), dtype=float32)


In [45]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [46]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


In [47]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
print("adder_node: ", adder_node)

adder_node:  Tensor("add_4:0", dtype=float32)


In [48]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


In [49]:
add_and_triple = adder_node * 3.
print("add_and_triple: ", add_and_triple)
print(sess.run(add_and_triple, {a: 3, b:4.5}))

add_and_triple:  Tensor("mul_5:0", dtype=float32)
22.5


In [50]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b
print("linear_model: ", linear_model)

linear_model:  Tensor("add_5:0", dtype=float32)


In [51]:
init = tf.global_variables_initializer()
sess.run(init)

In [52]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [53]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


In [54]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


In [55]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [56]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# Complete program

In [57]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# tf.contrib.learn

In [58]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

In [59]:
# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

In [60]:
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_model_dir': 'E:\\Tmp\\tmpcv7ua81k', '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000183BE11C7B8>, '_tf_random_seed': None, '_environment': 'local', '_evaluation_master': '', '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0, '_task_type': None, '_master': '', '_save_checkpoints_secs': 600}


In [61]:
# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

In [62]:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train,
                                              batch_size=4,
                                              num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x":x_eval}, y_eval, batch_size=4, num_epochs=1000)

In [63]:
# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into E:\Tmp\tmpcv7ua81k\model.ckpt.
INFO:tensorflow:loss = 4.75, step = 1
INFO:tensorflow:global_step/sec: 815.695
INFO:tensorflow:loss = 0.0753087, step = 101 (0.123 sec)
INFO:tensorflow:global_step/sec: 850.923
INFO:tensorflow:loss = 0.0118723, step = 201 (0.118 sec)
INFO:tensorflow:global_step/sec: 814.971
INFO:tensorflow:loss = 0.00241324, step = 301 (0.126 sec)
INFO:tensorflow:global_step/sec: 944.263
INFO:tensorflow:loss = 0.000662253, step = 401 (0.103 sec)
INFO:tensorflow:global_step/sec: 766.884
INFO:tensorflow:loss = 0.000143263, step = 501 (0.130 sec)
INFO:tensorflow:g

LinearRegressor(params={'joint_weights': False, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x00000183BDFFEE80>, 'gradient_clip_norm': None, 'optimizer': None})

In [64]:
# Here we evaluate how well our model did.
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-19-09:48:24
INFO:tensorflow:Restoring parameters from E:\Tmp\tmpcv7ua81k\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-06-19-09:48:26
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.56098e-07
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-19-09:48:26

# A custom model

In [65]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

In [66]:
estimator = tf.contrib.learn.Estimator(model_fn=model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_model_dir': 'E:\\Tmp\\tmp695li6ab', '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000183BE38B8D0>, '_tf_random_seed': None, '_environment': 'local', '_evaluation_master': '', '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0, '_task_type': None, '_master': '', '_save_checkpoints_secs': 600}


In [67]:
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

In [68]:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)

In [69]:
# train
estimator.fit(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into E:\Tmp\tmp695li6ab\model.ckpt.
INFO:tensorflow:loss = 261.014675655, step = 1
INFO:tensorflow:global_step/sec: 857.969
INFO:tensorflow:loss = 0.0536195217261, step = 101 (0.119 sec)
INFO:tensorflow:global_step/sec: 997.245
INFO:tensorflow:loss = 0.00382696945587, step = 201 (0.098 sec)
INFO:tensorflow:global_step/sec: 865.831
INFO:tensorflow:loss = 0.000514967500268, step = 301 (0.115 sec)
INFO:tensorflow:global_step/sec: 868.98
INFO:tensorflow:loss = 5.21957981448e-05, step = 401 (0.115 sec)
INFO:tensorflow:global_step/sec: 988.248
INFO:tensorflow:loss = 2.19188691296e-06, step = 501 (0.101 sec)
INFO:tensorflow:global_step/sec: 853.252
INFO:tensorflow:loss = 6.11022549728e-08, step = 601 (0.117 sec)
INFO:tensorflow:global_step/sec: 860.218
INFO:tensorflow:loss = 2.52703603967e-08, step = 701 (0.116 sec)
INFO:tensorflow:global_step/sec: 997.518
INFO:tensorflow:loss = 2.70866273434e-09, step = 801 

Estimator(params=None)

In [70]:
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Starting evaluation at 2017-06-19-09:48:32
INFO:tensorflow:Restoring parameters from E:\Tmp\tmp695li6ab\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-06-19-09:48:33
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 2.33855e-11
INFO:tensorflow:Starting evaluation at 2017-06-19-09:48:34
INFO:tensorflow:Restoring parameters from E:\Tmp\tmp695li6ab\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-06-19-09:48:35
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101006
train loss: {'global_step': 1000, 'loss': 2.3385545e-11}
eval loss: {'global_step': 1000, 'loss': 0.010100563}
